<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

**Загружаем необходимые библиотеки и инструменты**

In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

**Загрузим и изучим датасет**

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('D:/Яндекс Практикум/23. Машинное обучение для текстов/toxic_comments.csv')

In [3]:
data

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [6]:
data.duplicated().sum()

0

In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Данные загружены и изучены. Дубликаты и пропуски не обнаружены. Наш целевой признак - столбец `toxic` - содержит приблизительно 10% негативных комментариев, т.е. наблюдается высокий дисбалланс классов.

**Очистка текста**

Напишем функцию очистки текста и функцию лемматизации текста. Лемматизированный текст сохраним в новом столбце `processed_text`

In [8]:
def clean_text(text):
    text = text.lower()
    regular = r'[^\w ]'
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    text = re.sub(regular, ' ', text)
    text = re.sub(regular_url, ' ', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)',' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
 
def lemmatize(text):
    text = clean_text(text)
    tokens = nltk.word_tokenize(text)
    stop_text = [word for word in tokens if word not in stop_words]
    tagged_tokens = nltk.pos_tag(stop_text)
    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(token, pos)
        lemmatized_tokens.append(lemma)
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text
 
data['processed_text'] = data['text'].apply(lemmatize)

In [9]:
data.head()

,Unnamed: 0,text,toxic,processed_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


**Разделим данные на два датасета - тренировочный и тестовый**

In [10]:
train_data, test_data = train_test_split(data,train_size=0.75,random_state=42)

In [11]:
train_data.shape

(119469, 4)

In [12]:
test_data.shape

(39823, 4)

## Обучение

**Обучим три модели - Логистическую регрессию, Дерево решений и модель градиентного бустинга LightGBM. Для этого с помощью кроссвалидации разобьём тренировочный датасет на тренировочную, валидационную и тестовую выборки, подберём гиперпараметры с помощью GridSearchCV. Получим предсказания. Оценим качество предсказаний с помощью метрик F1-мера. Затем выберем лучшую модель для испытания на тестовых данных**

**Модель Логистическая регрессия**

In [13]:
X = train_data['processed_text']
y = train_data['toxic']
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []
best_test_params = 0
best_params_gs_lr = []

for trainval_index, test_index in tqdm(cv.split(X, y), total=cv.get_n_splits()):
    X_trainval, X_test = X.iloc[trainval_index], X.iloc[test_index]
    y_trainval, y_test = y.iloc[trainval_index], y.iloc[test_index]
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.25, random_state=42)

    tfidf_vect = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True, 
                                 strip_accents=None, lowercase=False, preprocessor=None)
    
    lr = LogisticRegression(random_state=42)

    param_grid = {'lr__penalty': ['l1', 'l2'],
                   'lr__C': [1.0, 5.0, 10.0],
                   'lr__solver': ['lbfgs', 'liblinear']}

    pipeline = Pipeline([
        ('tfidf', tfidf_vect),
        ('lr', lr)
    ])

    gs_lr_tfidf = GridSearchCV(pipeline, param_grid,
                               scoring='f1',
                               cv=5,
                               verbose=1,
                               n_jobs=-1)

    gs_lr_tfidf.fit(X_train, y_train)

    best_params = gs_lr_tfidf.best_params_
    print("Best Parameters:", best_params)

    y_pred = gs_lr_tfidf.predict(X_val)

    f1_score_val = f1_score(y_val, y_pred)
    print("f1_score:", f1_score_val)

    print(classification_report(y_val, y_pred))

    y_pred_test = gs_lr_tfidf.predict(X_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    print("Test f1_score:", f1_score_test)
    print(classification_report(y_test, y_pred_test))

    results.append(f1_score_val)

    if f1_score_test > best_test_params:
        best_test_params = f1_score_test
        best_params_gs_lr = gs_lr_tfidf.best_params_
        gs_lr_best_score_tfidf = gs_lr_tfidf.best_score_

average_f1_score = np.mean(results)
print("Average f1_score:", average_f1_score)
print(gs_lr_tfidf.best_score_)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'lr__C': 5.0, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
f1_score: 0.7792148854095756
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21512
           1       0.85      0.72      0.78      2382

    accuracy                           0.96     23894
   macro avg       0.91      0.85      0.88     23894
weighted avg       0.96      0.96      0.96     23894



 20%|████████████████▊                                                                   | 1/5 [01:04<04:18, 64.71s/it]

Test f1_score: 0.7741935483870966
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.84      0.71      0.77      2434

    accuracy                           0.96     23894
   macro avg       0.91      0.85      0.88     23894
weighted avg       0.96      0.96      0.96     23894

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'lr__C': 5.0, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
f1_score: 0.7840501792114696
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21461
           1       0.86      0.72      0.78      2433

    accuracy                           0.96     23894
   macro avg       0.92      0.85      0.88     23894
weighted avg       0.96      0.96      0.96     23894



 40%|█████████████████████████████████▌                                                  | 2/5 [02:10<03:15, 65.07s/it]

...

Test f1_score: 0.7693685624720108
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.85      0.71      0.77      2434

    accuracy                           0.96     23894
   macro avg       0.91      0.85      0.87     23894
weighted avg       0.95      0.96      0.96     23894

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'lr__C': 5.0, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
f1_score: 0.7712243074173369
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     21471
           1       0.84      0.71      0.77      2423

    accuracy                           0.96     23894
   macro avg       0.90      0.85      0.87     23894
weighted avg       0.96      0.96      0.96     23894



 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [04:19<01:04, 64.70s/it]

Test f1_score: 0.7765649365114725
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.85      0.72      0.78      2434

    accuracy                           0.96     23894
   macro avg       0.91      0.85      0.88     23894
weighted avg       0.96      0.96      0.96     23894

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'lr__C': 5.0, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
f1_score: 0.7741793346552105
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21407
           1       0.86      0.71      0.77      2487

    accuracy                           0.96     23894
   macro avg       0.91      0.85      0.88     23894
weighted avg       0.96      0.96      0.96     23894



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:22<00:00, 64.55s/it]

Test f1_score: 0.7818506928922664
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.86      0.72      0.78      2433

    accuracy                           0.96     23893
   macro avg       0.91      0.85      0.88     23893
weighted avg       0.96      0.96      0.96     23893

Average f1_score: 0.773799106749522
0.7740624285691966


In [14]:
print("Модель Логистическая регрессия")
print("F1 лучшей модели:", gs_lr_best_score_tfidf)
print("Параметры лучшей модели:", best_params_gs_lr)

Модель Логистическая регрессия
F1 лучшей модели: 0.7740624285691966
Параметры лучшей модели: {'lr__C': 5.0, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}


**Модель Дерево решений**

In [15]:
X = train_data['processed_text']
y = train_data['toxic']

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []
best_test_params = 0
best_params_gs_dt = []

for trainval_index, test_index in tqdm(cv.split(X, y), total=cv.get_n_splits()):
    X_trainval, X_test = X.iloc[trainval_index], X.iloc[test_index]
    y_trainval, y_test = y.iloc[trainval_index], y.iloc[test_index]

    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.25, random_state=42)

    tfidf_vect = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True, 
                                 strip_accents=None, lowercase=False, preprocessor=None)
    X_train_tfidf = tfidf_vect.fit_transform(X_train)
    X_val_tfidf = tfidf_vect.transform(X_val)
    X_test_tfidf = tfidf_vect.transform(X_test)

    dt = DecisionTreeClassifier(random_state=42)

    param_grid = {'dt__max_depth' : np.arange(76, 80)}
    
    pipeline = Pipeline([
        ('tfidf', tfidf_vect),
        ('dt', dt)
    ])

    gs_dt_tfidf = GridSearchCV(pipeline, param_grid,
                               scoring='f1',
                               cv=5,
                               verbose=1,
                               n_jobs=-1)

    gs_dt_tfidf.fit(X_train, y_train)

    best_params = gs_dt_tfidf.best_params_
    print("Best Parameters:", best_params)

    y_pred = gs_dt_tfidf.predict(X_val)

    f1_score_val = f1_score(y_val, y_pred)
    print("f1_score:", f1_score_val)

    print(classification_report(y_val, y_pred))

    y_pred_test = gs_dt_tfidf.predict(X_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    print("Test f1_score:", f1_score_test)
    print(classification_report(y_test, y_pred_test))

    results.append(f1_score_val)

    if f1_score_test > best_test_params:
        best_test_params = f1_score_test
        best_params_gs_dt = gs_dt_tfidf.best_params_
        gs_dt_best_score_tfidf = gs_dt_tfidf.best_score_

average_f1_score = np.mean(results)
print("Average f1_score:", average_f1_score)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Parameters: {'dt__max_depth': 76}
f1_score: 0.7174424071462153
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21512
           1       0.82      0.64      0.72      2382

    accuracy                           0.95     23894
   macro avg       0.89      0.81      0.84     23894
weighted avg       0.95      0.95      0.95     23894



 20%|████████████████▌                                                                  | 1/5 [04:54<19:36, 294.02s/it]

Test f1_score: 0.7094972067039105
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21460
           1       0.82      0.63      0.71      2434

    accuracy                           0.95     23894
   macro avg       0.89      0.81      0.84     23894
weighted avg       0.94      0.95      0.94     23894

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Parameters: {'dt__max_depth': 76}
f1_score: 0.7153385536619069
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21461
           1       0.81      0.64      0.72      2433

    accuracy                           0.95     23894
   macro avg       0.89      0.81      0.84     23894
weighted avg       0.95      0.95      0.95     23894



 40%|█████████████████████████████████▏                                                 | 2/5 [09:53<14:50, 296.98s/it]

...

Test f1_score: 0.7218799908738306
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21460
           1       0.81      0.65      0.72      2434

    accuracy                           0.95     23894
   macro avg       0.89      0.82      0.85     23894
weighted avg       0.95      0.95      0.95     23894

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Parameters: {'dt__max_depth': 79}
f1_score: 0.7191216834400732
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21471
           1       0.81      0.65      0.72      2423

    accuracy                           0.95     23894
   macro avg       0.88      0.82      0.85     23894
weighted avg       0.95      0.95      0.95     23894



 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [20:03<05:02, 302.20s/it]

Test f1_score: 0.7237442922374429
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21460
           1       0.81      0.65      0.72      2434

    accuracy                           0.95     23894
   macro avg       0.89      0.82      0.85     23894
weighted avg       0.95      0.95      0.95     23894

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Parameters: {'dt__max_depth': 77}
f1_score: 0.7102593010146562
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21407
           1       0.81      0.63      0.71      2487

    accuracy                           0.95     23894
   macro avg       0.88      0.81      0.84     23894
weighted avg       0.94      0.95      0.94     23894



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [25:19<00:00, 303.85s/it]

Test f1_score: 0.7178661761324442
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     21460
           1       0.81      0.64      0.72      2433

    accuracy                           0.95     23893
   macro avg       0.89      0.81      0.84     23893
weighted avg       0.95      0.95      0.95     23893

Average f1_score: 0.7138811277704598


In [16]:
print("Модель Дерево решений")
print("F1 лучшей модели:", gs_dt_best_score_tfidf)
print("Параметры лучшей модели:", best_params_gs_dt)

Модель Дерево решений
F1 лучшей модели: 0.7200899657452525
Параметры лучшей модели: {'dt__max_depth': 79}


**Модель градиентного бустинга LightGBM**

In [17]:
X = train_data['processed_text']
y = train_data['toxic']

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []
best_test_params = 0
best_params_gs_lgb = []

for trainval_index, test_index in tqdm(cv.split(X, y), total=cv.get_n_splits()):
    X_trainval, X_test = X.iloc[trainval_index], X.iloc[test_index]
    y_trainval, y_test = y.iloc[trainval_index], y.iloc[test_index]

    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.25, random_state=42)

    tfidf_vect = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True, 
                                 strip_accents=None, lowercase=False, preprocessor=None)
    X_train_tfidf = tfidf_vect.fit_transform(X_train)
    X_val_tfidf = tfidf_vect.transform(X_val)
    X_test_tfidf = tfidf_vect.transform(X_test)

    lgb_estimator = lgb.LGBMClassifier(task='train', boosting_type='gbdt', objective='regression', verbose=-1)

    param_grid_lgb = {'lg__num_leaves': [71, 81, 91],
                      'lg__learning_rate': [0.01, 0.1, 1.0]}
    
    pipeline = Pipeline([
        ('tfidf', tfidf_vect),
        ('lg', lgb_estimator)
    ])

    gs_lgb_tfidf = GridSearchCV(pipeline, 
                               param_grid_lgb,
                               scoring='f1',
                               cv=5,
                               verbose=1,
                               n_jobs=-1,
                               error_score='raise')

    gs_lgb_tfidf.fit(X_train, y_train)

    best_params = gs_lgb_tfidf.best_params_
    print("Best Parameters:", best_params)

    y_pred = gs_lgb_tfidf.predict(X_val)

    f1_score_val = f1_score(y_val, y_pred)
    print("f1_score:", f1_score_val)

    print(classification_report(y_val, y_pred))

    y_pred_test = gs_lgb_tfidf.predict(X_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    print("Test f1_score:", f1_score_test)
    print(classification_report(y_test, y_pred_test))

    results.append(f1_score_val)

    if f1_score_test > best_test_params:
        best_test_params = f1_score_test
        best_params_gs_lgb = gs_lgb_tfidf.best_params_
        gs_lgb_best_score_tfidf = gs_lgb_tfidf.best_score_

average_f1_score = np.mean(results)
print("Average f1_score:", average_f1_score)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'lg__learning_rate': 0.1, 'lg__num_leaves': 91}
f1_score: 0.7594762232942798
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21512
           1       0.84      0.69      0.76      2382

    accuracy                           0.96     23894
   macro avg       0.90      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894



 20%|████████████████▌                                                                  | 1/5 [10:46<43:07, 646.80s/it]

Test f1_score: 0.7579042457091237
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.84      0.69      0.76      2434

    accuracy                           0.96     23894
   macro avg       0.90      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'lg__learning_rate': 0.1, 'lg__num_leaves': 71}
f1_score: 0.7572727272727273
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21461
           1       0.85      0.68      0.76      2433

    accuracy                           0.96     23894
   macro avg       0.91      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894



 40%|█████████████████████████████████▏                                                 | 2/5 [21:47<32:45, 655.12s/it]

...

Test f1_score: 0.7602523659305993
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.84      0.69      0.76      2434

    accuracy                           0.96     23894
   macro avg       0.90      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'lg__learning_rate': 0.1, 'lg__num_leaves': 81}
f1_score: 0.7644144144144144
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21471
           1       0.84      0.70      0.76      2423

    accuracy                           0.96     23894
   macro avg       0.90      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894



 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [43:33<10:50, 650.82s/it]

Test f1_score: 0.7606527651858568
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     21460
           1       0.85      0.69      0.76      2434

    accuracy                           0.96     23894
   macro avg       0.91      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'lg__learning_rate': 0.1, 'lg__num_leaves': 71}
f1_score: 0.7605884975479268
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     21407
           1       0.85      0.69      0.76      2487

    accuracy                           0.96     23894
   macro avg       0.91      0.84      0.87     23894
weighted avg       0.95      0.96      0.95     23894



100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [54:03<00:00, 648.66s/it]

Test f1_score: 0.7559161595672751
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     21460
           1       0.84      0.69      0.76      2433

    accuracy                           0.95     23893
   macro avg       0.90      0.84      0.87     23893
weighted avg       0.95      0.95      0.95     23893

Average f1_score: 0.7579873960993361


In [18]:
print("Модель градиентного бустинга LightGBM")
print("F1 лучшей модели:", gs_lgb_best_score_tfidf)
print("Параметры лучшей модели:", best_params_gs_lgb)

Модель градиентного бустинга LightGBM
F1 лучшей модели: 0.7540526744791002
Параметры лучшей модели: {'lg__learning_rate': 0.1, 'lg__num_leaves': 71}


Лучшие показатели - метрику качества F1 - 0.77 и время обучения - 5 мин. 22 сек. показала модель Логистическая регрессия. Выбираем её для тестирования.

**Выделим из тестового датафрейма целевой и обучающий признаки**

In [19]:
X_test = test_data['processed_text']
y_test = test_data['toxic']

**TF-IDF-векторизация полного тренировочного и тестового датафреймов**

In [20]:
X_tfidf = tfidf_vect.fit_transform(X)
X_test_tfidf = tfidf_vect.transform(X_test)

**Обучим модель Логистическая регрессия с подобранными гиперпараметрами и получим предсказание на тестовых данных**

In [21]:
best_model = LogisticRegression(penalty='l1', C=5.0, solver='liblinear').fit(X_tfidf, y)
predicts = best_model.predict(X_test_tfidf)
print("Метрика F1 на тестовых данных:", f1_score(y_test, predicts))

Метрика F1 на тестовых данных: 0.7903334683407588


## Выводы

**Нам была поставлена задача - обучить модель классифицировать комментарии на позитивные и негативные. Для её достижения нами были поставлены и выполнены следующие задачи:**
- загружены необходимые библиотеки и инструменты;
- загружен и изучеен предоставленный датафрейм;
- произведена обработка и подготовка данных для обучения моделей: 
    - проверка на дубликаты, 
    - очистка текста от лишних символов,
    - лемматизация текста,
    - разделение данных на тренировочный и тестовый датасеты;
- были выбраны три модели для обучения - Логистическая регрессия, Дерево решений и модель градиентного бустинга LightGBM;
- произвели обучение моделей с подбором гиперпараметров;
- для тестирования выбрали лучшую модель по метрике F1 - Логистическая регрессия;
- получили предсказание модели на тестовом датафрейме.

**Во время подготовки данных для обучения выполнили очистку и лемматизацию текста, полученный текст сохранили в новом столбце `processed_text`, разбили исходный датасет на два - тренировочный и тестовый.**

**Для обучения были выбраны три модели - Логистическая регрессия, Дерево решений и модель градиентного бустинга LightGBM. Для обучения каждой модели с помощью кроссвалидации разбили тренировочный датасет на тренировочную, валидационную и тестовую выборки, подобрали гиперпараметры с помощью GridSearchCV и получили предсказания. Лучшую метрику F1 - 0.77 и минимальное время обучения - 5 мин. 22 сек. показала модель Логистическая регрессия. Поэтому для тестирования была выбрана эта модель.**

**По результатам проверки модели на тестовых данных убедились, что модель является работоспособной и надёжной, имеет высокую  F1-меру - 0.79, при минимально допустимом значении 0.75 и может применяться для классифицикации комментариев пользователей на позитивные и негативные.**

**Поставленная задача выполнена.**